## Exercise SEP Developer Test

USGS-SAC-25-12553801-DE-RM
SUPERVISORY COMPUTER SCIENTIST
GS-1550-14

Please contact me if you have questions regarding this submission. I may be reached at email, mjranda@gmail.com, or by cell at (360) 921-8778.

While I have done several projects using Python, I am more proficient developing web applications with PHP and JavaScript/HTML. Also I have not used Colab and Jupyter notebooks before this project. So this was an interesting project but may not be in-line with current standards. Cells will need to be run in order, and MySQL will need to be running in the environment.

In order to keep things simple, I have included test passwords in the variables cell. I understand this would not be acceptible in almost all environments.

### Original Requirements

This test is designed to assess your ability to design and develop a project from a limited set of goals, provided below. We will evaluate your project based on its fulfillment of the outlined objectives, code efficiency, readability, and adherence to best practices. Additionally, we will evaluate the structure and content of your GitHub repository according to standard practices.

1. Please perform all coding within the Google Colab environment: http://colab.research.google.com
    * Please store your project in a GitHub repository, so we can fork and run it in Colab.
        * Additionally, please create a standard GitHub repository with the expected files and directory structure.    
    * Please note that Colab doesn't automatically include additional files. To avoid potential issues, we recommend only including files that the evaluators already have access to.    
    * The evaluators will use the files SEP01.mseed, SEP02.mseed, and SEP03.mseed, which can be found in this GitHub project.
2. Import the mseed files
3. Create a database with proper normal form and constraints
4. Import the data into the newly created database
5. Create a visualization for the data. The visualization can be static but must include:    
    * A title
    * Text section displaying metadata about the miniseed data
    * Helicorder-style charts of the miniseed data (These may be built from the database data or the mseed files)
    * A map with icons indicating station locations based on the miniseed data (station lat and lon can be found on iris' API)
    * Add to the map additional stations found on IRIS' data API. Stations of interest include HOA and SUG
        * Data API information can be found at http://service.iris.edu/fdsnws/station/1/



# Install MySQL and download python packages (if needed)

In [ ]:
# Install MySQL
!apt-get -y install mysql-server
!service mysql start
# Colab may have these already, but just in case.
!pip install mysql-connector-python
!pip install obspy
!pip install folium

# Variables

In [ ]:
# In a production environment - some of these variables would be kept outside of this script in a "secrets" structure.

# These are the source URL's for the mseed data.
mseed_urls = [
    "https://github.com/CHIGHT-USGS/SeismicData/raw/main/SEP/SEP01.mseed",
    "https://github.com/CHIGHT-USGS/SeismicData/raw/main/SEP/SEP02.mseed",
    "https://github.com/CHIGHT-USGS/SeismicData/raw/main/SEP/SEP03.mseed"
]

# And the station data sources from IRIS
iris_urls = [
    "https://service.iris.edu/fdsnws/station/1/query?net=CC&sta=HOA&level=station&format=xml&includecomments=true&nodata=404",
    "https://service.iris.edu/fdsnws/station/1/query?net=CC&sta=SEP&level=station&format=xml&includecomments=true&nodata=404",
    "https://service.iris.edu/fdsnws/station/1/query?net=CC&sta=SUG&level=station&format=xml&includecomments=true&nodata=404"
]

# Database values.
mysql_host = 'localhost'
mysql_dbname = 'randall_code_test'
mysql_user = 'randalltest'
mysql_password = 'randalltest'

print('Variables loaded.')

# Create MySQL User
(assuming root account does not require a password on fresh MySQL install - would run MySQL secure installation scripts on production server).

In [ ]:
!mysql -u root --execute "create user if not exists '{mysql_user}'@'{mysql_host}' identified by '{mysql_password}'"
!mysql -u root --execute "grant all privileges on *.* to '{mysql_user}'@'{mysql_host}'"
!mysql -u root --execute "flush privileges"
print("User created")

# Import (download) Mseed Files

In [ ]:
from obspy import read
import requests
import pathlib

for url in mseed_urls:

    # Extract the file name from the url, if it is present on the file system, continue to the next url.
    file_name = url.split("/")[-1]
    if pathlib.Path(file_name).exists():
        print(f"{file_name} is already downloaded")
        continue

    # File was not present on the file system, download from remote server.
    response = requests.get(url)
    if response.status_code == 200:
        try:
            with open(file_name, 'wb') as file:
                file.write(response.content)
            print(f"{file_name} downloaded successfully")
        except Exception as e:
            print(f"Unable to download {url}: {e}")
            continue
    else:
        print(f"Failed to download file from {url}. Status code: {response.status_code}")
        continue


# Create a database with proper normal form and constraints

In [ ]:
import mysql.connector
conn = mysql.connector.connect(user=mysql_user, password=mysql_password, host=mysql_host)
cursor = conn.cursor()

print(f"CREATE DATABASE IF NOT EXISTS {mysql_dbname}")
cursor.execute(f"CREATE DATABASE IF NOT EXISTS {mysql_dbname}")

print(f"CREATE TABLE IF NOT EXISTS {mysql_dbname}.network")
cursor.execute(f"""
CREATE TABLE IF NOT EXISTS {mysql_dbname}.network (
  network_id INT UNSIGNED NOT NULL AUTO_INCREMENT,
  network_cd CHAR(2) NOT NULL,
  PRIMARY KEY (network_id),
  UNIQUE INDEX network_cd_unique (network_cd)
  )
""")

print(f"CREATE TABLE IF NOT EXISTS {mysql_dbname}.station")
cursor.execute(f"""
CREATE TABLE IF NOT EXISTS {mysql_dbname}.station (
  station_id INT UNSIGNED NOT NULL AUTO_INCREMENT,
  network_id INT UNSIGNED NOT NULL,
  station_cd CHAR(10) NOT NULL,
  latitude DECIMAL(9,6),
  longitude DECIMAL(9,6),
  elevation_meters DECIMAL(6,2),
  PRIMARY KEY (station_id),
  INDEX station_cd_idx (station_cd),
  INDEX station_fk1_idx (network_id),
  CONSTRAINT station_fk1
    FOREIGN KEY (network_id)
    REFERENCES {mysql_dbname}.network (network_id)
    )
""")

print(f"CREATE TABLE IF NOT EXISTS {mysql_dbname}.channel")
cursor.execute(f"""
CREATE TABLE IF NOT EXISTS {mysql_dbname}.channel (
  channel_id INT UNSIGNED NOT NULL AUTO_INCREMENT,
  station_id INT UNSIGNED NOT NULL,
  channel CHAR(4),
  PRIMARY KEY (channel_id),
  INDEX channel_idx (channel),
  INDEX channel_fk1_idx (station_id),
  CONSTRAINT channel_fk1
    FOREIGN KEY (station_id)
    REFERENCES {mysql_dbname}.station (station_id)
)
""")

print(f"CREATE TABLE IF NOT EXISTS {mysql_dbname}.channel_data")
cursor.execute(f"""
CREATE TABLE IF NOT EXISTS {mysql_dbname}.channel_data (
  channel_data_id INT UNSIGNED NOT NULL AUTO_INCREMENT,
  channel_id INT UNSIGNED NOT NULL,
  start_time TIMESTAMP(6),
  end_time TIMESTAMP(6),
  sampling_rate INT,
  serialized_trace LONGBLOB NULL,
  PRIMARY KEY (channel_data_id),
  INDEX channel_data_fk1_idx (channel_id),
  UNIQUE INDEX `channel_data_udx` (`channel_id` ASC, `start_time` ASC, `end_time`),
  CONSTRAINT channel_data_fk1
    FOREIGN KEY (channel_id)
    REFERENCES {mysql_dbname}.channel (channel_id)
)
""")

cursor.close()
conn.close()

# Import the data into the newly created database

In [ ]:
from obspy import read
import requests
import pathlib
import mysql.connector
import pickle

conn = mysql.connector.connect(user=mysql_user, password=mysql_password, host=mysql_host, database=mysql_dbname)
cursor = conn.cursor()

###############################################################################
### MySQL database functions.
###############################################################################

# Get the network_id value from the database for provided network_cd. If nothing is found,
# create a new record and return the new network_id.
def get_network_id(network_cd):

    network_id = None
    try:
        # Try selecting a record.
        cursor.execute("SELECT network_id FROM network WHERE network_cd = %s", (network_cd,))
        row = cursor.fetchone()

        # If nothing was selected, then insert a value and return the rowid, which is the network_id.
        # If something was selected, then just return the network_id
        if row == None:
            print("Creating network record %s" % network_cd)
            cursor.execute("INSERT INTO network (network_cd) VALUES (%s)", (network_cd,))
            conn.commit()
            network_id = cursor.lastrowid
        else:
            network_id = row[0]

    except mysql.connector.Error as err:
        print(err)

    finally:
        return network_id

# Get the station_id value from the database for provided station information. If nothing is found,
# create a new record and return the new station_id.
def get_station_id(station_cd, network_id):

    station_id = None
    try:
        # Try selecting a record.
        cursor.execute("SELECT station_id FROM station WHERE station_cd = %s AND network_id = %s", (station_cd, network_id,))
        row = cursor.fetchone()

        # If nothing was selected, then insert a value and return the rowid, which is the station_id.
        # If something was selected, then just return the station_id
        if row == None:
            print("Creating station record %s" % station_cd)
            cursor.execute("INSERT INTO station (station_cd, network_id) VALUES (%s, %s)", (station_cd, network_id))
            conn.commit()
            station_id = cursor.lastrowid
        else:
            station_id = row[0]

    except mysql.connector.Error as err:
        print(err)

    finally:
        return station_id

# Get the channel_id value from the database for provided information. If nothing is found,
# create a new record and return the new channel_id.
def get_channel_id(station_id, channel):

    channel_id = None
    try:
        # Try selecting a record.
        cursor.execute("SELECT channel_id FROM channel WHERE station_id = %s AND channel = %s",
                       (station_id, channel)
                      )
        row = cursor.fetchone()

        # If nothing was selected, then insert a value and return the rowid, which is the channel_id.
        # If something was selected, then just return the channel_id
        if row == None:
            print("Creating channel record %s" % channel)
            cursor.execute(
                            "INSERT INTO channel (station_id, channel) VALUES (%s, %s)",
                           (station_id, channel)
                          )
            conn.commit()
            channel_id = cursor.lastrowid
        else:
            channel_id = row[0]

    except mysql.connector.Error as err:
        print(err)

    finally:
        return channel_id

# Get the channel_id value from the database for provided information. If nothing is found,
# create a new record and return the new channel_id.
def get_channel_data_id(channel_id, start_time, end_time, sampling_rate, obspy_trace_serialized):

    channel_data_id = None
    try:
        # Try selecting a record.
        cursor.execute(
                        """
                        SELECT channel_data_id FROM channel_data
                        WHERE channel_id = %s AND start_time = FROM_UNIXTIME(%s) AND end_time = FROM_UNIXTIME(%s)
                        """,
                       (channel_id, start_time.timestamp, end_time.timestamp)
                      )
        row = cursor.fetchone()

        # If nothing was selected, then insert a value and return the rowid, which is the channel_data_id.
        # If something was selected, then just return the channel_data_id
        if row == None:
            print("Creating channel_data record %s, %s - %s" % (channel_id, start_time, end_time))
            cursor.execute(
                        """
                        INSERT INTO channel_data (channel_id, start_time, end_time, sampling_rate, serialized_trace)
                        VALUES (%s, FROM_UNIXTIME(%s), FROM_UNIXTIME(%s), %s, %s)
                        """,
                       (channel_id, start_time.timestamp, end_time.timestamp, sampling_rate, obspy_trace_serialized)
                      )
            conn.commit()
            channel_data_id = cursor.lastrowid
        else:
            channel_data_id = row[0]

    except mysql.connector.Error as err:
        print(err)

    finally:
        return channel_data_id

# Look in the file contents
for url in mseed_urls:

    file_name = url.split("/")[-1]
    print(f"\n#### Processing file {file_name} ####\n")
    obspy_stream = read(file_name)
    for obspy_trace in obspy_stream:

        # print(obspy_trace.stats)
        # for k, v in sorted(obspy_trace.stats.mseed.items()):
        #     print("'%s': %s" % (k, str(v)))
        # obspy_stream.plot(color='gray', tick_format='%I:%M %p',
        #     starttime=obspy_trace.stats.starttime,
        #     #endtime=obspy_trace.stats.starttime+20)
        #     endtime=obspy_trace.stats.endtime)

        network_id = get_network_id(obspy_trace.stats.network)
        station_id = get_station_id(obspy_trace.stats.station, network_id)
        channel_id = get_channel_id(station_id, obspy_trace.stats.channel)
        obspy_trace_serialized = pickle.dumps(obspy_trace)
        get_channel_data_id(channel_id, obspy_trace.stats.starttime, obspy_trace.stats.endtime, obspy_trace.stats.sampling_rate, obspy_trace_serialized)

cursor.close()
conn.close()

# Add Station Data from IRIS API
Add to the map additional stations found on IRIS' data API. Stations of interest include HOA and SUG. SEP lat/long and other metadata is also available here.

In [ ]:
import requests
import xml.etree.ElementTree as ET
import mysql.connector

conn = mysql.connector.connect(user=mysql_user, password=mysql_password, host=mysql_host, database=mysql_dbname)
cursor = conn.cursor()

# Get the network_id value from the database for provided network_cd. If nothing is found,
# create a new record and return the new network_id.
def get_network_id(network_cd):

    network_id = None
    try:
        # Try selecting a record.
        cursor.execute("SELECT network_id FROM network WHERE network_cd = %s", (network_cd,))
        row = cursor.fetchone()

        # If nothing was selected, then insert a value and return the rowid, which is the network_id.
        # If something was selected, then just return the network_id
        if row == None:
            print("Creating network record %s" % network_cd)
            cursor.execute("INSERT INTO network (network_cd) VALUES (%s)", (network_cd,))
            conn.commit()
            network_id = cursor.lastrowid
        else:
            network_id = row[0]

    except mysql.connector.Error as err:
        print(err)

    finally:
        return network_id

# Get the station_id value from the database for provided station information. If nothing is found,
# create a new record and return the new station_id. Returns the station_id if no errors.
def create_or_update_station(station_dict):

    station_id = None
    try:
        # Try selecting records.
        cursor.execute(
                    """
                    SELECT station_id, network_id, station_cd, IFNULL(latitude, 0), IFNULL(longitude, 0), IFNULL(elevation_meters, 0)
                    FROM station WHERE station_cd = %s AND network_id = %s
                    """,
                   (station_dict["station_cd"], station_dict["network_id"],)
                    )
        row = cursor.fetchone()

        # If nothing was selected, then insert a value and return the rowid, which is the station_id.
        # If something was selected, then just return the station_id
        if row == None:

            print("Creating station record %s" % station_dict["station_cd"])
            cursor.execute(
                            "INSERT INTO station (station_cd, network_id, latitude, longitude, elevation_meters) VALUES (%s, %s, %s, %s, %s)",
                           (station_dict["station_cd"], station_dict["network_id"], station_dict["latitude"], station_dict["longitude"], station_dict["elevation_meters"])
                          )
            conn.commit()
            station_id = cursor.lastrowid

        elif float(station_dict["latitude"]) != float(row[3]) or float(station_dict["longitude"]) != float(row[4]) or float(station_dict["elevation_meters"]) != float(row[5]):

            station_id = row[0]
            print("Updating station record %s" % station_dict["station_cd"])
            cursor.execute(
                            "UPDATE station SET latitude = %s, longitude = %s, elevation_meters = %s WHERE station_cd = %s AND network_id = %s",
                            (station_dict["latitude"], station_dict["longitude"], station_dict["elevation_meters"], station_dict["station_cd"], station_dict["network_id"])
                          )
            conn.commit()

        else:

            station_id = row[0]

    except mysql.connector.Error as err:
        print(err)

    finally:
        return station_id

def load_xml_from_url(url):

    try:
        response = requests.get(url)
        if response.status_code == 200:
            xml_doc = ET.fromstring(response.content)
            return xml_doc
        else:
            raise Exception(f"Failed to retrieve the XML data. Status code: {response.status_code}")
    except Exception as e:
        print(f"An error occurred: {e}")

for url in iris_urls:

    print("\n### Processing XML From %s" % url)
    xml_doc = load_xml_from_url(url)
    if xml_doc is not None:

        station_data = {
            "network_id": None,
            "station_cd": None,
            "latitude": None,
            "longitude": None,
            "elevation_meters": None
        }

        for child in xml_doc.iter():
            if 'Network' in child.tag.split("}"):
                station_data["network_id"] = get_network_id(child.attrib['code'])
            if 'Station' in child.tag.split("}"):
                station_data["station_cd"] = child.attrib['code']
            if 'Latitude' in child.tag.split("}"):
                station_data["latitude"] = child.text
            if 'Longitude' in child.tag.split("}"):
                station_data["longitude"] = child.text
            if 'Elevation' in child.tag.split("}"):
                station_data["elevation_meters"] = child.text

        if station_data["network_id"] is not None and station_data["station_cd"] is not None:
            print(station_data);
            create_or_update_station(station_data)

cursor.close()
conn.close()

# A Title and Text section displaying metadata about the miniseed data¶

In [ ]:
from IPython.display import display, HTML
import mysql.connector

summaries = None

try:

    conn = mysql.connector.connect(user=mysql_user, password=mysql_password, host=mysql_host, database=mysql_dbname)
    cursor = conn.cursor()

    cursor.execute(
                """
                SELECT
                	n.network_cd, s.station_cd,
                	c.channel, MIN(cd.start_time), MAX(cd.end_time), COUNT(*)
                FROM
                	network n
                		JOIN station s ON s.network_id = n.network_id
                			LEFT JOIN channel c ON c.station_id = s.station_id
                				LEFT JOIN channel_data cd ON cd.channel_id = c.channel_id
                GROUP BY
                	n.network_cd, s.station_cd, c.channel
                """
                )

    # In a production environment, with significant data, would use a where clause with a parameterized query to limit results.
    # If a map driven interface, using the lat/lon extents is a good way to query data along with using an event handler to
    # pull fresh data from API's as the user interacts with the map.
    summaries = cursor.fetchall()

    cursor.close()
    conn.close()

except mysql.connector.Error as err:
    print(err)

html = """
    <style>
        h1 {
          margin-top: 0;
        }
        .container {
            border: solid 1px gray;
            margin: 1rem;
            padding: 1rem;
            border-radius: 5px;
        }
    </style>
    <div class="container">
    <h1>Available Data</h1>
    """

html += """
  <table>
    <thead>
      <tr>
        <th>Network</th><th>Station</th><th>Channel</th>
        <th>Start</th><th>End</th><th>Traces</th>
      </tr>
    </thead>
    <tbody>
    """

for summary in summaries:
    if summary[3] is not None:
      html += f"""
              <tr>
                <td>{summary[0]}</td><td>{summary[1]}</td><td>{summary[2]}</td>
                <td>{summary[3]}</td><td>{summary[4]}</td><td>{summary[5]}</td>
              </tr>
              """
    else:
      html += f"""
              <tr>
                <td>{summary[0]}</td><td>{summary[1]}</td>
                <td colspan="4">Miniseed data not loaded.</td>
              </tr>
              """

html += "</tbody></table>"

###

html += "<h2>Plot data was pulled from these sources:</h2>"

for mseed_url in mseed_urls:
    html += mseed_url + "<br/>"

###

html += "<h2>Additional station data was pull from these sources:</h2>"
for iris_url in iris_urls:
    html += iris_url + "<br/>"

###

html += "</div>"

display(HTML(html))



# Helicorder-style charts of the miniseed data (built from the database data)

In [ ]:
import mysql.connector
import pickle
from obspy import read

channels = None
conn = mysql.connector.connect(user=mysql_user, password=mysql_password, host=mysql_host, database=mysql_dbname)
cursor = conn.cursor()

def get_mseed_data_from_mysql(channel_data_id):
    mseed_data = None
    try:
        cursor.execute("SELECT serialized_trace FROM channel_data WHERE channel_data_id = %s", (channel_data_id,))
        item = cursor.fetchone()
        if item is not None:
            mseed_data = item[0]
    except mysql.connector.Error as err:
        print(err)
    finally:
        return mseed_data

try:
    cursor.execute(
                """
                SELECT
                    cd.channel_data_id,
                	n.network_cd, s.station_cd, s.latitude, s.longitude, s.elevation_meters,
                	c.channel, cd.start_time, cd.end_time, cd.sampling_rate
                FROM
                	network n
                		JOIN station s ON s.network_id = n.network_id
                			JOIN channel c ON c.station_id = s.station_id
                				JOIN channel_data cd ON cd.channel_id = c.channel_id
                ORDER BY
                	s.station_cd, c.channel, cd.start_time
                """
                )
    # In a production environment, with significant data, we would use a where clause, or parameterized query to limit results.
    channels = cursor.fetchall()

except mysql.connector.Error as err:
    print(err)

for channel_item in channels:
    mseed_data = get_mseed_data_from_mysql(channel_item[0])
    obspy_trace = pickle.loads(mseed_data)
    obspy_trace.plot()

cursor.close()
conn.close()

# Map with icons indicating station locations based on the miniseed data including SEP, HOA and SUG

In [ ]:
import folium
import mysql.connector

stations = None

# Get stations from database.

conn = mysql.connector.connect(user=mysql_user, password=mysql_password, host=mysql_host, database=mysql_dbname)
cursor = conn.cursor()

try:
    # Try selecting a record.
    cursor.execute(
                """
                SELECT n.network_cd, s.station_cd, s.latitude, s.longitude, s.elevation_meters
                FROM   network n JOIN station s ON s.network_id = n.network_id
                ORDER BY s.station_cd
                """
                )
    stations = cursor.fetchall()

except mysql.connector.Error as err:
    print(err)

cursor.close()
conn.close()

# Create a map centered on a specific location
map = folium.Map(location=[46.241780, -122.191830], zoom_start=12)

for station in stations:

    folium.Marker(
        location = [station[2], station[3]],
        popup = f"{station[0]}.{station[1]}",
        icon = folium.Icon(color="red", icon="info-sign")
    ).add_to(map)

# Display Map
map